# Step 1. Get the LLM for the correction request

In [1]:
import openai
import os
from dotenv import load_dotenv
import json
import pandas as pd
import time

### A first look at the texts to be corrected

In [2]:
df = pd.read_parquet("../data/cleaned-latam-xix.parquet")

'''La publicacion del Oso se harà dos veces cada se mana, y constará de un pliego en cuarto ; 
ofreciendo à mas sus redactores, dar los gravados oportunos, siempre que loexija el asuntode 
que trate. Redactado por un Num. 8. TEMA del Periodico. POLITICA MILITAR. OCTAVA SESION. 
Abierta la sesion á las dore y un minuto de la noche , 25 de Febrero de 1845 , con asistencia 
de todos los Señores Representantes, se leyó y aprobó la acta de la Asamblea anterior , 
ménos en lo tocante à la torre del Convento de Santo Domingo, punto que quedó para ventilarse 
en mejor ocasion. En seguida se dió cuenta de una nota del Ejecutivo , referente à que urjía 
la necesidad de organizar un Ejército ; pues decia el Excmo. Decano: - "Un poder sin bayonetas 
vale tanto como un cero puesto á la izquierda."'''
print(df.loc[0, "text"])

La publicacion del Oso se harà dos veces cada se mana, y constará de un pliego en cuarto ; ofreciendo à mas sus redactores, dar los gravados oportunos, siempre que loexija el asuntode que trate. Redactado por un Num. 8. TEMA del Periodico. POLITICA MILITAR. OCTAVA SESION. Abierta la sesion á las dore y un minuto de la noche , 25 de Febrero de 1845 , con asistencia de todos los Señores Representantes, se leyó y aprobó la acta de la Asamblea anterior , ménos en lo tocante à la torre del Convento de Santo Domingo, punto que quedó para ventilarse en mejor ocasion. En seguida se dió cuenta de una nota del Ejecutivo , referente à que urjía la necesidad de organizar un Ejército ; pues decia el Excmo. Decano: - "Un poder sin bayonetas vale tanto como un cero puesto á la izquierda."


## 1. Prepare the LLM API Client and request parameters

In this case, the LLM is GPT in it's **GPT 3.5 Turbo** version

In [3]:
load_dotenv('./gpt35.env')
openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_API_VERSION")
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_ENDPOINT")
engine=os.getenv("OPENAI_IMPLEMENTATION")
model=os.getenv("MODEL_NAME")
model

'gpt-3.5-turbo'

In [4]:
def request(prompt, max_tokens=500, temperature=0):
    """Request a completion from the OpenAI API.
    :param prompt: The prompt to send to the API
    :param max_tokens: The maximum number of tokens in the output
    :param temperature: The degree of randomness in the output
    :return response: The response from the API
    :return usage: The count of token usage from the API { "input", "output" }
    """
    try:
        response = openai.ChatCompletion.create(
            api_key=api_key,
            api_base=api_base,
            engine=engine,
            max_tokens=max_tokens,
            temperature=temperature,
            messages=[{"role": "user", "content": prompt}],
        )
        finish_reason = response["choices"][0]["finish_reason"]
        rsp = "" if finish_reason == "content_filter" else response["choices"][0]["message"]["content"]
        return rsp, { "input": response["usage"]["prompt_tokens"], "output": response["usage"]["completion_tokens"] }, finish_reason
    except Exception as e:
        response = ""
        usage = {"input": 0, "output": 0}
        if f"{e}".startswith("The response was filtered due to the prompt triggering Azure OpenAI's content management policy"):
            finish_reason = "content_filter"
        else:
            if "Max retries exceeded with url" in f"{e}":
                time.sleep(60)
                return request(prompt, max_tokens, temperature) # retry after 60 seconds
            finish_reason = f"ERROR [{type(e).__name__}]: {e}"
        return response, usage, finish_reason

Define the prompt to send with the text:

In [5]:
prompt_start = 'Dado el texto entre ```, retorna únicamente el texto corrigiendo los errores ortográficos sin cambiar la gramática:\n```\n'
# text
prompt_end = '\n```'
gen_prompt = lambda text: f"{prompt_start}{text}{prompt_end}"

To recover the information from the last execution and avoid the lost of data if an error occurs:

In [6]:
RESPONSES_FILE = "./responsesLatam.json"

r = {"data":[], "checkpoint": 0, "input_tokens": 0, "output_tokens": 0}
if os.path.exists(RESPONSES_FILE):
    with open(RESPONSES_FILE, "r") as f:
        r = json.load(f)
else:
    with open(RESPONSES_FILE, "w") as f:
        f.write(json.dumps(r, indent=4))

In [7]:
assert r['checkpoint'] == len(r['data']), "Checkpoint does not match with corrected texts"
print(f"Done {r['checkpoint']}/{len(df)} ({100*r['checkpoint']/len(df):.2f}%)")

Done 10176/10176 (100.00%)


## 2. Send the requests to the API and store them periodically

In [8]:
for text in df.loc[r['checkpoint']:, "text"]:
    prompt = gen_prompt(text)
    print(f"------------------------------- {r['checkpoint']} -----------------------------------")
    print(prompt)
    response, usage, finish_reason = request(prompt)
    print(response)
    r["input_tokens"] += usage["input"]
    r["output_tokens"] += usage["output"]

    r["data"].append({
        "text": text,
        "resp": response,
        "finish_reason": finish_reason
    })

    r["checkpoint"] += 1

    if r['checkpoint'] % 10 == 0:
        with open(RESPONSES_FILE, "w") as f:
            f.write(json.dumps(r, indent=4))
        print(f"SAVED ({r['checkpoint']})")

with open(RESPONSES_FILE, "w") as f:
    f.write(json.dumps(r, indent=4))

There will be empty responses due to errors when sending the request. Some of them are due to connection issues, and others due to the OpenAI's content management policy. In this case, the request, can be run manually when there are only a few cases.

In [9]:
print("The following requests must be repeated manually (due to OpenAI's content filter or an error):\n")
for i,e in enumerate(r["data"]):
    if e["finish_reason"] == "content_filter" or e["finish_reason"].startswith("ERROR"):
        print(f"------------------------------- {i} -----------------------------------")
        print(gen_prompt(e['text']))

# put the responses in the dictionary and run the next cell

In [10]:
manually_req_responses = {
    269: '''175 para llamar la atención del Sr. Intendente de Policía sobre esa cruel y mal entendida economía que adoptan siempre los rejoneadores. Estos, en vez de entrar a la plaza en caballos que por su brío y buena rienda puedan librarse y salvar a sus jinetes de una muerte casi cierta, se presentan en caballos que más parecen perros galgos que caballos. Los entregan a las astas del toro a propósito y después de mil y mil fatigas, presentan al público un espectáculo no solo asqueroso, sino hasta bárbaro y cruel. ¡Cuántas veces no hemos visto esas pobres bestias, con los intestinos fuera, correteando y regando bajo las piernas de un bravo toro, con su sangre toda la plaza! La Intendencia debe ser muy severa a este respecto, y negar el premio que tan cruel y ferozmente exigen estos bárbaros. ¿Qué les importa, en efecto, sacrificar un caballo cuando lo tienen pagado a costa de la vida del mismo animal? ¿Qué mérito puede presentar esta clase de espectáculos cuando en ellos ya no hace alarde el lidiador de la destreza que debe? Esperamos pues que el Sr. Intendente corrija tan feroz abuso y si es posible, castigue al que no cumpla con el primero de los deberes, la humanidad. AVISOS Suplicamos a nuestros lectores disimulen, que no haya salido este número con la exactitud que los demás, pues tres oficiales de los que trabajaban en nuestro periódico se han despedido sin darnos el tiempo necesario para reemplazarlos. Los versos que insertamos a continuación los reimprimimos hoy, a petición de diez y siete de nuestros SS. escritores.''',
}

for i,e in enumerate(r["data"]):
    if e["finish_reason"] == "content_filter" or e["finish_reason"].startswith("ERROR"):
        print(f"------------------------------- {i} -----------------------------------")
        print(r["data"][i]['text'])
        if i in manually_req_responses:
            r["data"][i]["resp"] = manually_req_responses[i]
            r["data"][i]["finish_reason"] = "stop"
            print(r["data"][i]["resp"])
            print(f"Index {i} OK")
        else:
            print(f"Index {i} NOT FOUND")

with open(RESPONSES_FILE, "w") as f:
    f.write(json.dumps(r, indent=4))